In [2]:
import os
import sys
from dateutil.parser import parse
import pandas as pd
import numpy as np
import datetime
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sqlalchemy import create_engine
plt.style.use('fivethirtyeight')
import warnings

In [3]:
# 基本信息
DB_OUT_HOST = '172.16.170.81'
DB_OUT_PORT = 3307
DB_OUT_USER = 'liwenjie'
DB_OUT_PWD = 'liwenjie333'
DB_OUT_NAME = 'data_center'


engine = create_engine('mysql+pymysql://%(user)s:%(pwd)s@%(host)s:%(port)s/%(dbname)s?charset=utf8' % {'user': DB_OUT_USER, 'pwd': DB_OUT_PWD, 'host': DB_OUT_HOST, 'dbname': DB_OUT_NAME, 'port':DB_OUT_PORT},
                                       encoding='utf-8')
conn = engine.connect()
consum = pd.read_csv('../data/consumption.txt')
aimstudent = consum['student_id'].drop_duplicates()

# 基本信息
df_basicinfo = pd.read_sql_table('t_basicinfo', conn)

In [4]:
# 门禁
df_access = pd.read_csv('../data/access.txt')
df_access = df_access[df_access['student_id'].isin(df_basicinfo['student_id'])]
df_access['pass_time'] = pd.to_datetime(df_access.pass_time)
df_access = df_access.rename({'pass_time':'brush_time','direction':'trans_type'}, axis=1)
df_access['card_id'] = -1
df_access['category'] = 'dorm'
df_access['device_name'] = '-1'
df_access['device_id'] = -1
df_access['amount'] = 0
df_access['remained_amount'] = 0

In [5]:
# 消费
consum = pd.read_csv('../data/consumption.txt')
consum.drop(['id'], axis=1, inplace=True)
consum.dropna(subset=['place', 'student_id', 'brush_time'], inplace=True)
consum.fillna('', inplace=True)
consum['brush_time'] = pd.to_datetime(consum.brush_time)
consum['timeslot'] = '0'
consum = consum[consum['student_id'].isin(df_basicinfo['student_id'])]

In [6]:
# 合并
consum = pd.concat([consum, df_access])
consum = consum.sort_values(['brush_time'])
consum = consum.drop(['id', 'campus'], axis=1)

/home/liwenjie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


In [7]:
# 地点处理
pattern = [
        '电子科技大学_(\w{3})\dF.+',
        'null_(\w+)',
        '沙河校区_(\w+)',
        '电子科技大学_(.+)POS.+',
        '电子科技大学_(.+)收费.+',
        '(.+)_.+'  # general
    ]

def splitplace(x):
    res = x
    if x == 'dorm' or x =='library':
        return x
    for p in pattern:
        if re.match(p, x):
            res = re.findall(p, x)[0]
            return res
    # some error
    if x:
        print(x)
    return res

consum['place0'] = consum.place.apply(splitplace)
t, indexer = pd.factorize(consum.place0)
consum['placei'] = t

In [12]:
# 下次刷卡地点
# 添加下次刷卡地点标签
def label_next(x):
    if x.shape[0] == 1:
        label = [np.nan]
    else:
        # error: label last
        # label = [np.nan] + x['placei'][:-1].tolist()
        label = x['placei'][1:].tolist() + [np.nan]
    x['place_next'] = label
    return x

consum = consum.groupby(['student_id']).apply(label_next)
# consum = consum.dropna(subset=['place_next'])
# consum['place_next'] = consum['place_next'].apply(int)

MemoryError: 

In [10]:
consum.to_csv('../data/consum_access_feat5.csv', index=False)

In [11]:
consum

,amount,brush_time,card_id,category,device_id,device_name,place,remained_amount,student_id,timeslot,trans_type,place0,placei
0,0.00,2015-09-01 00:00:01,-1,dorm,-1,-1,dorm,0.00,2012019190013,NaN,出门,dorm,0
9,0.60,2015-09-01 00:00:11,269830,bath,000C8CB8,淋浴,本科26#楼_本科26#楼1F淋002,134.03,2014050205016,0,POS消费,本科26#楼,1
10,0.10,2015-09-01 00:00:11,343677,bath,000CC112,淋浴,电子科大沙河校区本部公寓17#_本部17#楼1F淋006,25.86,2014220102007,0,POS消费,电子科大沙河校区本部公寓17#,2
13,0.50,2015-09-01 00:00:12,151090,bath,000D7FD4,淋浴,电子科大沙河校区本部公寓12#_本部12,88.57,2012059170025,0,POS消费,电子科大沙河校区本部公寓12#,3
14,0.20,2015-09-01 00:00:12,207602,bath,000CC6E3,淋浴,电子科大沙河校区欣村公寓04#_欣村4,84.06,2013220402017,0,POS消费,电子科大沙河校区欣村公寓04#,4
15,0.30,2015-09-01 00:00:13,262159,bath,000D4D40,淋浴,电子科大沙河校区欣村公寓05#_欣村5#楼3F淋322房间,174.11,2013050207017,0,POS消费,电子科大沙河校区欣村公寓05#,5
16,0.30,2015-09-01 00:00:14,284255,bath,000C909C,淋浴,本科18#楼_本18楼1F淋006,105.28,2012019170004,0,POS消费,本科18#楼,6
20,1.30,2015-09-01 00:00:17,272382,bath,000BDB86,淋浴,电子科大沙河校区欣村公寓06#_欣村6#3F 000BDB86 淋浴17,81.60,2014220301012,0,POS消费,电子科大沙河校区欣村公寓06#,7
26,0.80,2015-09-01 00:00:30,330136,bath,000C9096,淋浴,本科24#楼_本科24#楼5F淋035,162.82,2012181010011,0,POS消费,本科24#楼,8
31,0.50,2015-09-01 00:00:36,344607,bath,000B70E6,淋浴,本科7#楼_本7#楼1F淋003,183.02,2012060090023,0,POS消费,本科7#楼,9
